<a href="https://colab.research.google.com/github/antonvandike/Algoritmos-Geneticos/blob/main/Libro_Eyal_Wirsansky/Handson_chapter_4_2_tsp_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Solucion del problema del agente de ventas viajero

###Configuraciones y definiciones

In [7]:
pip install deap

In [8]:
pip install tsplib95

In [9]:
## Paso 1: Modulos
from deap import base
from deap import creator
from deap import tools
from deap import algorithms

import random as rd
import array as arr

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import tsplib95
import elitism

ModuleNotFoundError: No module named 'elitism'

In [ ]:
## Paso 2: Crear una instancia de las caracteristicas del problema

with open('/content/bayg29.tsp') as f:
  text = f.read()

tsp = tsplib95.parse(text)

In [ ]:
## Paso 2: Parametros del problema

# Contenido del problema del agente viajero

POPULATION_SIZE = 300     # Tamaño de la poblacion de individuos
P_CROSSOVER     =  0.9    # Probabilidad de cruza
P_MUTATION      =  0.05    # Probabilidad de mutación

MAX_GENERATIONS = 200     # Maximo numero de generaciones (condición de termino)

In [ ]:
HALL_OF_FAME_SIZE = 30

In [ ]:
## Paso 3: Elementos aleatorios

RANDOM_SEED = 42          # Comunmente cuando se experimenta con un codigo
rd.seed(RANDOM_SEED)      # se corre el problema con los mismos parametros
                          # para repetir y compara resultados

In [ ]:
## Paso 4: Declarando el cromosoma
toolbox = base.Toolbox()
toolbox.register("randomOrder", rd.sample, range(tsp.dimension), tsp.dimension)

In [ ]:
## Paso 5: Crear la funcion de aptitud (FITNESS)

# Se indica maximizacion y un solo peso
creator.create('FitnessMin', base.Fitness, weights = (-1.0,))

In [ ]:
## Paso 6: Crear la funcion de  aptitud del individuo

creator.create("Individual", arr.array, typecode='i', fitness=creator.FitnessMin)

In [ ]:
## Paso 7: Declarar el operador para generador de individuos

toolbox.register("individualCreator", tools.initIterate, creator.Individual, toolbox.randomOrder)

In [ ]:
## Paso 8: Declarar el operador para generar la poblacion

toolbox.register("populationCreator", tools.initRepeat, list, toolbox.individualCreator)

In [ ]:
# Funcion para calcular la distancia
def TotalDistance(indices):

  distancia = 0
  for x in range(-1,len(indices)-1):

    edge = indices[x]+1,indices[x+1]+1
    distancia = distancia + tsp.get_weight(*edge)

  return distancia

In [ ]:
## Paso 9: Definir función para calcular la aptitud
def tpsDistance(individual):

  return TotalDistance(individual),  # return a tuple

In [ ]:
## Paso 10: Declarar el operador de evaluacion

toolbox.register("evaluate", tpsDistance)

In [ ]:
## Paso 11: Declarar los operadores geneticos

toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb = 1.0/tsp.dimension)

###Evolucionando la solución

In [ ]:
def TSP_problem():

    # create initial population (generation 0):
    population = toolbox.populationCreator(n=POPULATION_SIZE)

    # prepare the statistics object:
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", np.min)
    stats.register("avg", np.mean)

    # define the hall-of-fame object:
    hof = tools.HallOfFame(HALL_OF_FAME_SIZE)

    # perform the Genetic Algorithm flow with hof feature added:
    population, logbook = elitism.eaSimpleWithElitism(population, toolbox, cxpb=P_CROSSOVER, mutpb=P_MUTATION,
                                              ngen=MAX_GENERATIONS, stats=stats, halloffame=hof, verbose=True)

    # print best individual info:
    best = hof.items[0]
    print("-- Best Ever Individual = ", best)
    print("-- Solución exacta al problema bayg29 = ", 1610)
    print("-- Best Ever Fitness = ", best.fitness.values[0])


    # plot statistics:
    minFitnessValues, meanFitnessValues = logbook.select("min", "avg")
    plt.figure(2)
    sns.set_style("whitegrid")
    plt.plot(minFitnessValues, color='red')
    plt.plot(meanFitnessValues, color='green')
    plt.xlabel('Generation')
    plt.ylabel('Min / Average Fitness')
    plt.title('Min and Average fitness over Generations')
    plt.show()

    colores = ["*b-","*m-"]
    plt.figure(figsize=(12, 8))
    ax1 = plt.subplot(2,3,1)
    ax2 = plt.subplot(2,3,2)

    axes = [ax1, ax2]



    color = 0

    optimalSolution = [0, 27, 5, 11, 8, 25, 2, 28, 4, 20, 1, 19, 9, 3, 14, 17, 13, 16, 21, 10, 18, 24, 6, 22, 7, 26, 15, 12, 23]
    c = [best,optimalSolution]

    for y1 in range(2):
      for x in range(-1,tsp.dimension-1):
        a = tsp.display_data[c[y1][x]+1]
        b = tsp.display_data[c[y1][x+1]+1]
        axes[y1].plot([a[0],b[0]],[a[1],b[1]],colores[y1])

    # show both plots:
    plt.title('Comparación de la solución\nobtenida y la solución real')
    plt.show()




In [ ]:
TSP_problem()